In [4]:
import numpy as np
np.random.seed(42)
np.set_printoptions(suppress=True)
from scipy.optimize import least_squares, curve_fit
from matplotlib import pylab as plt
import pandas as pd
import glob
import time
from tqdm.notebook import tqdm
import os

import warnings
from scipy.optimize import OptimizeWarning
warnings.simplefilter("error", OptimizeWarning)

# import seaborn as sns
# sns.set()

# Defining Individual Functions for Each Parameterisation

### 1. Bazin - Old

In [5]:
bazinfeatures = ["a", "b", "t0", "tfall", "trise"]

def oldbazin(time, a, b, t0, tfall, trise):
    with np.errstate(over='ignore', invalid='ignore'):
        X = np.exp(-(time - t0) / tfall) / (1 + np.exp((time - t0) / trise))
        return a * X + b
    
def errfunc_oldbazin(params, time, flux, fluxerr):
    return abs(flux - oldbazin(time, *params))

def fit_scipy_oldbazin(time, flux, fluxerr):
    flux = np.asarray(flux)
    t0 = time[flux.argmax()] - time[0]
    guess = [0, 0, t0, 40, -5]

    result = least_squares(errfunc_oldbazin, guess, args=(time, flux, fluxerr), method='lm')

    return result.x

### 2. Bazin - New

In [6]:
bazinfeatures = ["a", "b", "t0", "tfall", "trise"]

def bazin(time, a, b, t0, tfall, trise):
    with np.errstate(over='ignore', invalid='ignore'):
        X = np.exp(-(time - t0) / tfall) / (1 + np.exp(-(time - t0) / trise))
        return a * X + b

def bazinr(time, a, b, t0, tfall, r):
    trise = tfall/r
    return bazin(time, a, b, t0, tfall, trise)
    
def errfunc_bazin(params, time, flux, fluxerr):
    return abs(flux - bazinr(time, *params)) / fluxerr

def fit_scipy_bazin(time, flux, fluxerr):
    flux = np.asarray(flux)
    imax = flux.argmax()
    flux_max = flux[imax]
    
    # Parameter guess
    a_guess = 2*flux_max
    b_guess = 0
    t0_guess = time[imax]
    
    tfall_guess = time[imax-2:imax+2].std()/2
    if np.isnan(tfall_guess):
        tfall_guess = time[imax-1:imax+1].std()/2
        if np.isnan(tfall_guess):
            tfall_guess=50
    if tfall_guess<1:
        tfall_guess=50

    r_guess = 2

    guess = [a_guess,b_guess,t0_guess,tfall_guess,r_guess]

    # Parameter bounds
    a_bounds = [1.e-3, np.inf]
    b_bounds = [-np.inf, np.inf]
    t0_bounds = [-0.5*time.max(), 1.5*time.max()]
    tfall_bounds = [1.e-3, np.inf]
    r_bounds = [1, np.inf]
    
    bounds = [[a_bounds[0], b_bounds[0], t0_bounds[0], tfall_bounds[0], r_bounds[0]],
              [a_bounds[1], b_bounds[1], t0_bounds[1], tfall_bounds[1], r_bounds[1]]]
    
    result = least_squares(errfunc_bazin, guess, args=(time, flux, fluxerr), method='trf', loss='linear',bounds=bounds)
    
    a_fit,b_fit,t0_fit,tfall_fit,r_fit = result.x
    trise_fit = tfall_fit/r_fit
    final_result = np.array([a_fit,b_fit,t0_fit,tfall_fit,trise_fit])
    
    return final_result

### 3. FRED

In [7]:
fredfeatures = ["tm", "r", "d", "Fm", "b"]

def fred(time, tm, r, d, Fm, b):
    t0 = 0 # No trigger time in our case and timestamps are arbitrary, so t0 = 0
    x = (time+t0)/(tm+t0)
    val = Fm * (x)**r * ((d)/(d+r) + (r)/(d+r) * (x)**(r+1) )**(-(r+d)/(r+1)) + b
    return val

def errfunc_fred(params, time, flux, fluxerr):
    return abs(flux - fred(time, *params))/ fluxerr


def fit_scipy_fred(time, flux, fluxerr):
    flux = np.asarray(flux)
    imax = flux.argmax()
    flux_max = flux[imax]
    
    tm_bounds = [-0.5*time.max(), 1.5*time.max()]
    r_bounds = [1.e-3,np.inf]
    d_bounds = [1.e-3,np.inf]
    Fm_bounds = [-np.inf,np.inf]
    b_bounds = [-np.inf,np.inf]
    
    # Parameter guess
    tm_guess = time[imax]
    r_guess = 30
    d_guess = 30
    Fm_guess = flux_max
    b_guess = time[0]
    
    guess = [tm_guess, r_guess, d_guess, Fm_guess, b_guess]

    result = least_squares(errfunc_fred, guess, args=(time, flux, fluxerr), method='trf', loss='linear',\
                                  bounds=([tm_bounds[0], r_bounds[0], d_bounds[0], Fm_bounds[0], b_bounds[0]],
                                          [tm_bounds[1], r_bounds[1], d_bounds[1], Fm_bounds[1], b_bounds[1]])
                          )
    
    return result.x

### 4. Karpenka

In [8]:
karpenkafeatures = ["a", "b", "t0", "tfall", "trise", "t1", "c"]

def karpenka(time, a, b, t0, tfall, trise, t1, c):
    val = a*(1+b*(time-t1)**2)* (np.exp(-(time-t0)/tfall))/(1+np.exp(-(time-t0)/trise))
    return val + c

def karpenkar(time, a, b, t0, tfall, r, t1, c):
    trise = tfall/r
    return karpenka(time, a, b, t0, tfall, trise, t1, c)

def errfunc_karpenka(params, time, flux, fluxerr):
    return abs(flux - karpenkar(time, *params)) / fluxerr

def fit_scipy_karpenka(time, flux, fluxerr):
    flux = np.asarray(flux)
    imax = flux.argmax()
    flux_max = flux[imax]
    
    # Parameter guess
    a_guess = 2*flux_max
    b_guess = 0
    t0_guess = time[imax]
    t1_guess = time[imax]
    c_guess = 0
    
    tfall_guess = time[imax-2:imax+2].std()/2
    if np.isnan(tfall_guess):
        tfall_guess = time[imax-1:imax+1].std()/2
        if np.isnan(tfall_guess):
            tfall_guess=50
    if tfall_guess<1:
        tfall_guess=50

    r_guess = 2

    guess = [a_guess,b_guess,t0_guess,tfall_guess,r_guess,t1_guess,c_guess]

    # Parameter bounds
    a_bounds = [1.e-3, np.inf]
    b_bounds = [-np.inf, np.inf]
    t0_bounds = [-0.5*time.max(), 1.5*time.max()]
    t1_bounds = [-0.5*time.max(), 1.5*time.max()]
    tfall_bounds = [1.e-3, np.inf]
    r_bounds = [1, np.inf]
    c_bounds = [-np.inf, np.inf]
    
    bounds = [[a_bounds[0], b_bounds[0], t0_bounds[0], tfall_bounds[0], r_bounds[0], t1_bounds[0], c_bounds[0]],
              [a_bounds[1], b_bounds[1], t0_bounds[1], tfall_bounds[1], r_bounds[1], t1_bounds[1], c_bounds[1]]]
    
    result = least_squares(errfunc_karpenka, guess, args=(time, flux, fluxerr), method='trf', loss='linear',bounds=bounds)
    
    a_fit,b_fit,t0_fit,tfall_fit,r_fit,t1_fit,c_fit = result.x
    trise_fit = tfall_fit/r_fit
    final_result = np.array([a_fit,b_fit,t0_fit,tfall_fit,trise_fit,t1_fit,c_fit])
    
    return final_result

### 5. Villar

In [9]:
villarfeatures = ["a", "beta", "t0", "gamma", "t_fall", "t_rise"]

def villarupper(time,a,beta,t0,t1,tfall,trise):
    val = (a + beta*(time-t0))/(1+np.exp(-(time-t0)/trise))
    return val

def villarlower(time,a,beta,t0,t1,tfall,trise):
    val = ((a+beta*(t1-t0))*np.exp(-(time-t1)/tfall))/(1+np.exp(-(time-t0)/trise))
    return val

def villar(time,a,beta,t0,gamma,tfall,trise):
    t1 = t0 + gamma
    val = np.piecewise(time, [time < t1, time >= t1], [lambda time: villarupper(time,a,beta,t0,t1,tfall,trise), lambda time: villarlower(time,a,beta,t0,t1,tfall,trise)])
    return val

def errfunc_villar(params, time, flux, fluxerr):
    return abs(flux - villar(time, *params))/ fluxerr

def fit_scipy_villar(time, flux, fluxerr):
    flux = np.asarray(flux)
    imax = flux.argmax()
    t0 = time[imax]
    max_flux = flux[imax]
    if max_flux>0:
        a_bounds = [max_flux / 3.0, max_flux * 3.0]
    else:
        a_bounds = [-np.inf, np.inf]
    beta_bounds = [0.0, 1.0]
    t0_bounds = [-0.5*time.max(), 1.5*time.max()]
    gamma_bounds = [1.0, 100.0]
    tfall_bounds = [1.0, 100.0]
    trise_bounds = [1.0, 100.0]

    
    a_guess = np.clip(1.5 * max_flux, a_bounds[0], a_bounds[1])
    beta_guess = 0
    t0_guess = np.clip(time[imax] * 2.0 / 3.0, t0_bounds[0], t0_bounds[1])
    gamma_guess = np.clip(time[imax], gamma_bounds[0], gamma_bounds[1])
    tfall_guess = 50
    trise_guess = 45
    
    guess = [a_guess, beta_guess, t0_guess, gamma_guess, tfall_guess, trise_guess]
    
    result = least_squares(errfunc_villar, guess, args=(time, flux, fluxerr), method='trf', loss='linear',\
                                  bounds=([a_bounds[0], beta_bounds[0], t0_bounds[0], gamma_bounds[0], tfall_bounds[0], trise_bounds[0]],
                                         [a_bounds[1], beta_bounds[1], t0_bounds[1], gamma_bounds[1], tfall_bounds[1], trise_bounds[1]])
                          )
    return result.x


### 6. ALERCE v1

In [10]:
alercev1features = ["a", "beta", "t0", "gamma", "t_fall", "t_rise"]

def upperfn(time,a,beta,t0,t1,tfall,trise):
    val = (a*(1 - beta*(time-t0)/(t1-t0)))/(1+np.exp(-(time-t0)/trise))
    return val
def lowerfn(time,a,beta,t0,t1,tfall,trise):
    val = (a*(1-beta)*np.exp(-(time-t1)/tfall))/(1+np.exp(-(time-t0)/trise))
    return val
def alercev1(time,a,beta,t0,gamma,tfall,trise):
    t1 = t0 + gamma
    val = np.piecewise(time, [time < t1, time >= t1], [lambda time: upperfn(time,a,beta,t0,t1,tfall,trise), lambda time: lowerfn(time,a,beta,t0,t1,tfall,trise)])
    return val

def errfunc_alercev1(params, time, flux, fluxerr):
    return abs(flux - alercev1(time, *params))/ fluxerr

def fit_scipy_alercev1(time, flux, fluxerr):
    flux = np.asarray(flux)
    imax = flux.argmax()
    t0 = time[imax]
    max_flux = flux[imax]
    if max_flux>0:
        a_bounds = [max_flux / 3.0, max_flux * 3.0]
    else:
        a_bounds = [-np.inf, np.inf]
    beta_bounds = [0.0, 1.0]
    t0_bounds = [-0.5*time.max(), 1.5*time.max()]
    gamma_bounds = [1.0, 100.0]
    tfall_bounds = [1.0, 100.0]
    trise_bounds = [1.0, 100.0]

    
    a_guess = np.clip(1.5 * max_flux, a_bounds[0], a_bounds[1])
    beta_guess = 0
    t0_guess = np.clip(time[imax] * 2.0 / 3.0, t0_bounds[0], t0_bounds[1])
    gamma_guess = np.clip(time[imax], gamma_bounds[0], gamma_bounds[1])
    tfall_guess = 50
    trise_guess = 45
    
    guess = [a_guess, beta_guess, t0_guess, gamma_guess, tfall_guess, trise_guess]
    
    result = least_squares(errfunc_alercev1, guess, args=(time, flux, fluxerr), method='trf', loss='linear',\
                                  bounds=([a_bounds[0], beta_bounds[0], t0_bounds[0], gamma_bounds[0], tfall_bounds[0], trise_bounds[0]],
                                         [a_bounds[1], beta_bounds[1], t0_bounds[1], gamma_bounds[1], tfall_bounds[1], trise_bounds[1]])
                          )
    return result.x


### 7. Sánchez-Sáez (ALERCE v2)
(NOTE: 3 Different implementations were tried for this earlier [here](https://github.com/siddharthchaini/comparing-parameterisations/blob/a2310de1b68eb47b38d671d70aadbcc839aad7af/2.%20fit%20and%20save%20all.ipynb). The following is the implementation that performed the best.

In [11]:
alercev2features = ["A", "t0", "gamma", "beta", "t_rise", "t_fall"]

def alercev2(times, A, t0, gamma, beta, t_rise, t_fall):
    sigmoid_factor = 1.0 / 3.0
    t1 = t0 + gamma

    sigmoid = 1.0 / (1.0 + np.exp(-sigmoid_factor * (times - t1)))
    den = 1 + np.exp(-(times - t0) / t_rise)
    flux = ((1 - beta) * np.exp(-(times - t1) / t_fall)
            * sigmoid
            + (1. - beta * (times - t0) / gamma)
            * (1 - sigmoid)) * A / den
    return flux

def fit_scipy_alercev2(times, fluxpsf, obs_errors):
    '''
    Adapted from:
    https://github.com/alercebroker/lc_classifier/blob/main/lc_classifier/features/extractors/sn_parametric_model_computer.py#L113
    '''
    # Parameter bounds
    argmax_fluxpsf = np.argmax(fluxpsf)
    max_fluxpsf = fluxpsf[argmax_fluxpsf]
    A_bounds = [max_fluxpsf / 5, max_fluxpsf * 5]
    t0_bounds = [-0.5*times.max(), times.max()]
    gamma_bounds = [1.0, 50]  # (1,50/100)
    beta_bounds = [0.0, 1.0]
    trise_bounds = [1.0, 100.0]
    tfall_bounds = [1.0, 100.0]
    
    
    # Parameter guess
    A_guess = np.clip(1.5 * max_fluxpsf, A_bounds[0], A_bounds[1])
    t0_guess = np.clip(times[argmax_fluxpsf] * 2.0 / 3.0, t0_bounds[0], t0_bounds[1])
    gamma_guess = np.clip(times[argmax_fluxpsf], gamma_bounds[0], gamma_bounds[1])
#     gamma_guess = 50
    beta_guess = 0
    trise_guess = np.clip(times[argmax_fluxpsf] / 2.0, trise_bounds[0], trise_bounds[1])
#     print(f"trise_gess = np.clip({times[argmax_fluxpsf] / 2.0},{trise_bounds[0]},{trise_bounds[1]})")
#     trise_guess = 45
    tfall_guess = 40
        
    p0 = [A_guess, t0_guess, gamma_guess,
          beta_guess, trise_guess, tfall_guess]
    # get parameters
    try:
        pout, pcov = curve_fit(
            f=alercev2,
            xdata=times,
            ydata=fluxpsf,
            p0=p0,
            sigma=5+obs_errors,
            bounds=[[A_bounds[0], t0_bounds[0], gamma_bounds[0], beta_bounds[0], trise_bounds[0], tfall_bounds[0]],
                    [A_bounds[1], t0_bounds[1], gamma_bounds[1], beta_bounds[1], trise_bounds[1], tfall_bounds[1]]],
            ftol=1e-8,
            # verbose=2
        )
    except (ValueError, RuntimeError, OptimizeWarning):
        try:
            # print('First fit of SPM failed. Attempting second fit.')
            pout, pcov = curve_fit(
                f=alercev2,
                xdata=times,
                ydata=fluxpsf,
                p0=p0,
                sigma=5+obs_errors,
                bounds=[[A_bounds[0], t0_bounds[0], gamma_bounds[0], beta_bounds[0], trise_bounds[0], tfall_bounds[0]],
                        [A_bounds[1], t0_bounds[1], gamma_bounds[1], beta_bounds[1], trise_bounds[1], tfall_bounds[1]]],
                ftol=0.1,
                # verbose=2
            )
        except (ValueError, RuntimeError, OptimizeWarning):
            # print('Two fits of SPM failed. Returning NaN.')
            pout = np.array([np.nan, np.nan, np.nan, np.nan, np.nan, np.nan])    
    return pout


## 8. Bazin - With NEWMOD (set defaults if value low)
### ADD LATER

In [12]:
# bazinfeatures = ["a", "b", "t0", "tfall", "trise"]

# def bazin(time, a, b, t0, tfall, trise):
#     with np.errstate(over='ignore', invalid='ignore'):
#         X = np.exp(-(time - t0) / tfall) / (1 + np.exp(-(time - t0) / trise))
#         return a * X + b

# def bazinr(time, a, b, t0, tfall, r):
#     trise = tfall/r
#     return bazin(time, a, b, t0, tfall, trise)
    
# def errfunc_bazin(params, time, flux, fluxerr):
#     return abs(flux - bazinr(time, *params)) / fluxerr

# def errfunc_bazin4pts(params, time, flux, fluxerr):
#     a,t0,tfall,r = params
#     b=0 #HARDCODED
#     return abs(flux - bazinr(time, a, b, t0, tfall, r)) / fluxerr

# def errfunc_bazin3pts(params, time, flux, fluxerr):
#     a,t0,tfall = params
#     b=0 #HARDCODED
#     trise=10 #HARDCODED
#     r = tfall/trise
#     return abs(flux - bazinr(time, a, b, t0, tfall, r)) / fluxerr

# def fit_scipy_bazinNEWMOD(time, flux, fluxerr):
#     flux = np.asarray(flux)
#     imax = flux.argmax()
#     flux_max = flux[imax]
    
#     # Parameter guess
#     a_guess = 2*flux_max
#     b_guess = 0
#     t0_guess = time[imax]
#     tfall_guess = time[imax-2:imax+2].std()/2
#     if np.isnan(tfall_guess):
#         tfall_guess = time[imax-1:imax+1].std()/2
#         if np.isnan(tfall_guess):
#             tfall_guess=50
#     if tfall_guess<1:
#         tfall_guess=50
#     r_guess = 2

        
#     # Parameter bounds
#     a_bounds = [1.e-3, np.inf]
#     b_bounds = [-np.inf, np.inf]        
#     t0_bounds = [-0.5*time.max(), 1.5*time.max()]
#     tfall_bounds = [1.e-3, np.inf]
#     r_bounds = [1, np.inf]

    
#     # Full fit
#     if len(flux)>=5:
#         guess = [a_guess,b_guess,t0_guess,tfall_guess,r_guess]
#         bounds = [[a_bounds[0], b_bounds[0], t0_bounds[0], tfall_bounds[0], r_bounds[0]],
#                   [a_bounds[1], b_bounds[1], t0_bounds[1], tfall_bounds[1], r_bounds[1]]]
#         result = least_squares(errfunc_bazin, guess, args=(time, flux, fluxerr), method='trf', loss='linear',bounds=bounds)
#         a_fit,b_fit,t0_fit,tfall_fit,r_fit = result.x
#         trise_fit = tfall_fit/r_fit
#     elif len(flux)==4:
#         b_fit=0 #HARDCODED
#         guess = [a_guess,t0_guess,tfall_guess,r_guess]
#         bounds = [[a_bounds[0], t0_bounds[0], tfall_bounds[0], r_bounds[0]],
#                   [a_bounds[1], t0_bounds[1], tfall_bounds[1], r_bounds[1]]]
#         result = least_squares(errfunc_bazin4pts, guess, args=(time, flux, fluxerr), method='trf', loss='linear',bounds=bounds)
#         a_fit,t0_fit,tfall_fit,r_fit = result.x
#         trise_fit = tfall_fit/r_fit
#     else:
#         b_fit=0 #HARDCODED
#         trise_fit=10 #HARDCODED
#         guess = [a_guess,t0_guess,tfall_guess]
#         tfall_bounds[0] = trise_fit # CHANGE tfall lower bound to ensure tfall>trise as no r>1 bound here
#         bounds = [[a_bounds[0], t0_bounds[0], tfall_bounds[0]],
#                   [a_bounds[1], t0_bounds[1], tfall_bounds[1]]]
#         result = least_squares(errfunc_bazin3pts, guess, args=(time, flux, fluxerr), method='trf', loss='linear',bounds=bounds)
#         a_fit,t0_fit,tfall_fit = result.x
    
#     final_result = np.array([a_fit,b_fit,t0_fit,tfall_fit,trise_fit])
    
#     return final_result

---

In [13]:
directoryls = glob.glob("subset_csv_data/*_days")

In [14]:
obsdaylist = []
for directory in directoryls:
    obsdaylist.append(int(directory.split("/")[-1].split("_")[0]))

In [15]:
# filels = glob.glob("csv_data/*.csv")

In [16]:
def runfit(filels, fit_scipy, featurename, csvname):
    
    failedfits = 0

    datalist = []

    for file in tqdm(filels,desc=csvname.split("/")[-1]):
        d = pd.read_csv(file)
        obid = int(file.split("/")[-1].split("_")[0])
        obclass = file.split("_")[-1].replace(".csv","")
        timels = []
        featurels = []
        for filt in "ugrizY":
            band = d['band']==filt 
            t = np.array(d[band]['mjd'])
            if len(t)==0:
                continue
            t = t-t[0]
            f = np.array(d[band]['flux'])
            fe = np.array(d[band]['fluxerr'])

            try:
                timestart = time.time()
                params_list = list(fit_scipy(t, f, fe))
                timestop = time.time()
                tottime = timestop-timestart
            except Exception as e:
#                 print(e)
                failedfits += 1
                params_list = [np.nan for x in featurename]
                tottime = np.nan
            featurels = featurels + params_list
            timels.append(tottime)
        timetaken = np.mean([x for x in timels if x == x])

        datalist.append([obid,obclass,timetaken]+featurels)

    collist = []
    for fil in "ugrizY":
        for f in featurename:
            collist.append(f"{f}_{fil}")

    df = pd.DataFrame(data=datalist,columns=["objid","objclass","timetaken"]+collist)

    if not os.path.isdir("subset_saved_fits/"):
        os.mkdir("subset_saved_fits/")
    
    df.to_csv(f"subset_saved_fits/{csvname}.csv",index=False)

    print(f"Failed fitting {failedfits}/{len(filels)*6} light curves")

In [17]:
if not os.path.isdir("subset_saved_fits/"):
    os.mkdir("subset_saved_fits/")

for obsday in obsdaylist:
    if not os.path.isdir(f"subset_saved_fits/{obsday}_days/"):
        os.mkdir(f"subset_saved_fits/{obsday}_days/")
    filels = glob.glob(f"subset_csv_data/{obsday}_days/*.csv")
    print(f"********** {obsday} DAYS **********")
    runfit(filels=filels, fit_scipy = fit_scipy_oldbazin, featurename=bazinfeatures,csvname=f"{obsday}_days/oldbazin")
    runfit(filels=filels, fit_scipy = fit_scipy_bazin, featurename=bazinfeatures,csvname=f"{obsday}_days/bazin")
    runfit(filels=filels, fit_scipy = fit_scipy_fred, featurename=fredfeatures,csvname=f"{obsday}_days/fred")    
    runfit(filels=filels, fit_scipy = fit_scipy_karpenka, featurename=karpenkafeatures,csvname=f"{obsday}_days/karpenka")    
    runfit(filels=filels, fit_scipy = fit_scipy_villar, featurename=villarfeatures,csvname=f"{obsday}_days/villar")    
    runfit(filels=filels, fit_scipy = fit_scipy_alercev1, featurename=alercev1features,csvname=f"{obsday}_days/alercev1")    
    runfit(filels=filels, fit_scipy = fit_scipy_alercev2, featurename=alercev2features,csvname=f"{obsday}_days/alercev2")    

********** 111 DAYS **********


oldbazin:   0%|          | 0/480 [00:00<?, ?it/s]

Failed fitting 0/2880 light curves


bazin:   0%|          | 0/480 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

Failed fitting 0/2880 light curves


alercev1:   0%|          | 0/480 [00:00<?, ?it/s]

Failed fitting 0/2880 light curves


alercev2:   0%|          | 0/480 [00:00<?, ?it/s]

Failed fitting 0/2880 light curves
